In [1]:
#Load required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [2]:
#Load clinical and gene expression data

#This dataset comprises gene expression of 173 cancer genes in 2,433 primary tumours from the METABRIC dataset
clin = pd.read_csv("brca_metabric/data_clinical_sample.txt",sep='\t')
exp = pd.read_csv("/Users/emilyjin/Desktop/DS exercise/brca_metabric/data_mrna_agilent_microarray_zscores_ref_all_samples.txt", sep='\t')

#Obtain list of mutation driver genes and pam50 breast cancer subtypes from the METABRIC study
mut_genes = pd.read_csv("brca_metabric/ncomms11479-s2.csv")
pam50 = pd.read_csv("brca_metabric/data_clinical_patient.txt",sep='\t')

In [3]:
#Output the first ten rows of clinical and gene expression data 
clin.head(10)

,#Patient Identifier,Sample Identifier,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Neoplasm Histologic Grade,Oncotree Code,PR Status,Sample Type,Tumor Size,Tumor Stage,TMB (nonsynonymous)
0,#Identifier to uniquely specify a patient.,A unique sample identifier.,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Numeric value to express the degree of abnorma...,Oncotree Code,PR Status,"The type of sample (i.e., normal, primary, met...",Tumor size.,Tumor stage.,TMB (nonsynonymous)
1,#STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER
2,#1,1,1,1,1,1,1,1,1,1,1,1,1
3,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,ER_STATUS,HER2_STATUS,GRADE,ONCOTREE_CODE,PR_STATUS,SAMPLE_TYPE,TUMOR_SIZE,TUMOR_STAGE,TMB_NONSYNONYMOUS
4,MB-0000,MB-0000,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Negative,Primary,22,2,0
5,MB-0002,MB-0002,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,10,1,2.615035408
6,MB-0005,MB-0005,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,2,IDC,Positive,Primary,15,2,2.615035408
7,MB-0006,MB-0006,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Positive,Negative,2,MDLC,Positive,Primary,25,2,1.307517704
8,MB-0008,MB-0008,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Positive,Negative,3,MDLC,Positive,Primary,40,2,2.615035408
9,MB-0010,MB-0010,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,31,4,5.230070815


In [4]:
exp.head(10)

,Hugo_Symbol,Entrez_Gene_Id,MB-0362,MB-0346,MB-0386,MB-0574,MB-0185,MB-0503,MB-0641,MB-0201,...,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313,MB-4823
0,RERE,473,-0.7139,1.2266,-0.0053,-0.4399,-0.5958,0.4729,0.4974,-1.1900,...,-0.4596,1.8975,1.1120,1.1942,-1.7974,1.1339,0.0259,-0.3529,-1.2327,1.7217
1,RNF165,494470,-0.4606,0.3564,-0.6800,-1.0563,-0.0377,-0.6829,-0.2854,-0.4336,...,-1.0927,0.9103,-0.0023,-0.2898,3.5763,1.3429,0.5726,0.1731,0.5482,1.2239
2,PHF7,51533,-0.3325,-1.0617,0.2587,-0.2982,-1.2422,0.0558,-0.5011,-0.6418,...,-0.0725,0.7219,0.1402,0.8718,-0.9275,-0.0587,0.5240,-0.0311,4.4925,-0.2173
3,CIDEA,1149,-0.0129,-1.0394,3.2991,-0.2632,-1.0949,1.2628,2.0796,-0.8310,...,0.0679,-0.7126,-0.1523,-0.7593,-0.7141,-0.4324,-0.0336,-0.4003,2.4698,-0.7268
4,TENT2,167153,-0.7853,0.0337,-0.6649,2.1640,-0.2031,1.0304,0.6046,-1.7557,...,0.6400,-0.1102,1.2719,0.8178,-1.0301,0.6082,0.5608,2.4222,-3.2853,0.4181
5,SLC17A3,10786,0.7630,-0.5175,0.6719,-1.1036,-0.9250,-1.2571,0.2490,0.6489,...,0.1667,-0.0730,-0.1773,0.7417,0.0210,-0.3136,-0.6894,0.6773,0.9610,1.3746
6,SDS,10993,0.8752,-0.5610,0.1445,-0.0361,-0.6270,0.4277,-1.4874,-1.0626,...,0.7116,-1.4491,-1.2932,-0.2196,0.2432,-1.1075,-0.1793,1.5110,2.1707,-0.8606
7,ATP6V1C2,245973,-0.5499,3.6712,-0.2674,-0.4254,-1.1306,-0.6752,0.1474,1.4754,...,-0.5614,-0.1896,-0.3453,-0.8822,-0.2069,0.3955,-0.1489,-0.8557,0.5251,-0.2209
8,F3,2152,-1.3871,-1.1852,-1.0100,-1.1183,-1.2610,-0.9561,-1.2115,0.7501,...,-0.5435,-0.1091,0.1709,-1.2617,1.2137,2.4321,-0.0958,-0.0595,0.1038,1.8961
9,FAM71C,196472,0.2502,-0.8299,-1.4008,0.8459,0.1368,-0.4509,-1.9467,-1.8007,...,0.4445,0.3719,-0.2140,-0.1256,-0.4179,0.1945,-0.3498,-0.6682,-2.7623,-0.3821


In [5]:
mut_genes.head(10)

,#Supplementary Dataset 1 - Details of genes & mutations in this study,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,"#Genes names, positions and annotation transcr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HGNC_symbol,Chr,Start,End,Strand,Annotation_transcript,Number_mutations,Number_synonymous,Number_missense,Number_nonsense,Number_nonframeshift,Number_frameshift,Number_splice,Number_nonexonic,Homozygous_deletions,LOH,Amplifications,Amplifications_mutation,HD_inclusion,DoubleMuts
2,ACVRL1,12,52300702,52317645,+,ENST00000388922,72,7,12,1,0,0,0,45,0,257,3,0,Yes,13
3,AFF2,X,147581639,148082693,+,ENST00000370460,296,28,40,1,0,1,0,226,0,494,7,0,No,42
4,AGMO,7,15239443,15602140,-,ENST00000342526,117,11,24,1,2,2,3,74,0,233,10,0,Yes,32
5,AGTR2,X,115301458,115306725,+,ENST00000371906,40,0,14,0,0,0,0,26,0,481,2,0,No,14
6,AHNAK,11,62200516,62314832,-,ENST00000378024,387,82,237,5,3,0,0,55,0,509,8,0,No,219
7,AHNAK2,14,105403091,105445194,-,ENST00000333244,878,322,524,4,2,6,0,19,0,781,8,0,No,395
8,AKAP9,7,91569689,91740487,+,ENST00000356239,265,30,137,7,2,4,2,82,0,325,12,0,No,143
9,AKT1,14,105235187,105262580,-,ENST00000554581,193,17,96,0,2,0,0,78,0,571,10,2,No,98


In [6]:
pam50.head(10)

,#Patient Identifier,Lymph nodes examined positive,Nottingham prognostic index,Cellularity,Chemotherapy,Cohort,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,...,Overall Survival Status,Pam50 + Claudin-low subtype,3-Gene classifier subtype,Patient's Vital Status,Primary Tumor Laterality,Radio Therapy,Tumor Other Histologic Subtype,Type of Breast Surgery,Relapse Free Status,Relapse Free Status (Months)
0,#Identifier to uniquely specify a patient.,Number of lymphnodes positive,Nottingham prognostic index,Tumor Content,Chemotherapy.,Cohort.,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,...,Overall patient survival status.,Pam50 + Claudin-low subtype,3-Gene classifier subtype,The survival state of the person.,"For tumors in paired organs, designates the si...",Radio Therapy,Text to describe a tumor's histologic subtype ...,Type of Breast Surgery,Relapse Free Status (Recurred = cases where th...,Time until last follow-up or relapse (in months)
1,#STRING,STRING,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,...,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER
2,#1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,PATIENT_ID,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,...,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY,RFS_STATUS,RFS_MONTHS
4,MB-0000,10,6.044,NaN,NO,1,Positve,NEUTRAL,YES,Post,...,0:LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY,0:Not Recurred,138.65
5,MB-0002,0,4.02,High,NO,1,Positve,NEUTRAL,YES,Pre,...,0:LIVING,LumA,ER+/HER2- High Prolif,Living,Right,YES,Ductal/NST,BREAST CONSERVING,0:Not Recurred,83.52
6,MB-0005,1,4.03,High,YES,1,Positve,NEUTRAL,YES,Pre,...,1:DECEASED,LumB,NaN,Died of Disease,Right,NO,Ductal/NST,MASTECTOMY,1:Recurred,151.28
7,MB-0006,3,4.05,Moderate,YES,1,Positve,NEUTRAL,YES,Pre,...,0:LIVING,LumB,NaN,Living,Right,YES,Mixed,MASTECTOMY,0:Not Recurred,162.76
8,MB-0008,8,6.08,High,YES,1,Positve,NEUTRAL,YES,Post,...,1:DECEASED,LumB,ER+/HER2- High Prolif,Died of Disease,Right,YES,Mixed,MASTECTOMY,1:Recurred,18.55
9,MB-0010,0,4.062,Moderate,NO,1,Positve,NEUTRAL,YES,Post,...,1:DECEASED,LumB,ER+/HER2- High Prolif,Died of Disease,Left,YES,Ductal/NST,MASTECTOMY,1:Recurred,2.89


In [7]:
#Output further information about the data object 
clin.info()

print("\t")

pam50.info()

print("\t")

exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2513 entries, 0 to 2512
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   #Patient Identifier        2513 non-null   object
 1   Sample Identifier          2513 non-null   object
 2   Cancer Type                2513 non-null   object
 3   Cancer Type Detailed       2513 non-null   object
 4   ER Status                  2473 non-null   object
 5   HER2 Status                1984 non-null   object
 6   Neoplasm Histologic Grade  2392 non-null   object
 7   Oncotree Code              2513 non-null   object
 8   PR Status                  1984 non-null   object
 9   Sample Type                2513 non-null   object
 10  Tumor Size                 2364 non-null   object
 11  Tumor Stage                1792 non-null   object
 12  TMB (nonsynonymous)        2513 non-null   object
dtypes: object(13)
memory usage: 255.4+ KB
	
<class 'pandas.core.fra

In [8]:
#Remove unwanted columns and rows from clinical data 
clin1 = clin[["Sample Identifier", "ER Status", "HER2 Status", "PR Status"]]
rows = clin1.index[0:4]
clin1 = clin1.drop(rows)

#Removing rows with NA values in any columns
clin1 = clin1.dropna()

#Reset the numerical index column in the dataframe
clin1 = clin1.reset_index(drop=True)

clin1.head(10)

,Sample Identifier,ER Status,HER2 Status,PR Status
0,MB-0000,Positive,Negative,Negative
1,MB-0002,Positive,Negative,Positive
2,MB-0005,Positive,Negative,Positive
3,MB-0006,Positive,Negative,Positive
4,MB-0008,Positive,Negative,Positive
5,MB-0010,Positive,Negative,Positive
6,MB-0014,Positive,Negative,Positive
7,MB-0020,Negative,Negative,Negative
8,MB-0022,Positive,Negative,Negative
9,MB-0028,Positive,Negative,Negative


In [9]:
#Filter gene expression dataset to include mutation driver genes 

#Rename column name from list of mutation genes
mut_genes.rename(columns={'#Supplementary Dataset 1 - Details of genes & mutations in this study':'Hugo_Symbol'}, inplace=True)

#Remove unwanted rows and subset only the gene names column
mut_genes = mut_genes.iloc[2:,0]

mut_genes.head(10)

#Subset gene expression data to contain only mutation driver genes
exp_mut = pd.merge(exp, mut_genes, how='right', on='Hugo_Symbol').fillna(0)

exp_mut

,Hugo_Symbol,Entrez_Gene_Id,MB-0362,MB-0346,MB-0386,MB-0574,MB-0185,MB-0503,MB-0641,MB-0201,...,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313,MB-4823
0,ACVRL1,94.0,-0.5410,-0.3123,-0.0132,-0.7274,-1.1754,0.2561,2.0242,0.7213,...,-1.0633,-1.6927,0.2543,-1.2449,0.2091,0.2897,-0.1199,-2.0597,2.4153,-0.4894
1,AFF2,2334.0,-0.6422,-0.0426,-0.1310,-0.2717,-1.2299,0.2915,0.0248,0.5369,...,-0.3061,0.3954,1.2609,-0.9739,0.8172,1.3611,0.4086,0.1750,4.1895,-0.2543
2,AGMO,392636.0,-0.9923,-0.3136,0.0881,-0.5359,0.1550,-1.4104,1.7006,-0.0739,...,-0.5859,-1.0897,0.6400,0.5404,1.8769,0.7336,0.8889,2.5236,-0.3690,0.4056
3,AGTR2,186.0,0.3742,1.6764,0.2763,0.5059,0.2532,0.5655,2.2020,0.2421,...,1.1125,0.3824,0.4570,0.5675,-0.7437,0.3806,-0.5476,1.3890,-0.4971,0.2910
4,AHNAK,79026.0,-0.8728,-0.8483,0.9198,-0.4719,-0.8634,1.1923,0.7532,0.3335,...,-1.1885,-0.1764,0.0671,1.1090,0.0780,1.7488,0.9271,2.1139,-0.7401,-0.4300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,USH2A,7399.0,-0.5777,-0.0007,0.1022,0.3891,-0.6642,0.0336,-0.5083,1.1706,...,-0.3001,0.5691,0.7746,0.3250,0.9911,-0.1029,0.3451,1.4625,1.5421,1.8622
169,USP28,57646.0,-1.0270,-0.3306,-0.9569,-0.5684,-0.2926,-1.5118,0.0016,1.9597,...,1.0651,0.1628,0.5927,-1.1280,-0.0855,0.4521,1.3336,-1.1749,-0.7397,2.0093
170,USP9X,8239.0,0.2693,1.1136,-0.3321,1.8919,-1.1574,0.8636,-0.3306,-1.2128,...,-0.1091,-3.9190,-0.0594,-0.6085,1.3949,-0.4807,1.2854,-0.6935,-3.3095,-0.0714
171,UTRN,7402.0,0.7628,-0.7678,0.5739,0.0802,0.0751,-0.5426,0.3240,-0.1225,...,-0.8116,-0.4603,1.2000,-0.0310,0.4886,0.9076,-0.1727,0.2820,2.2526,0.8062


In [10]:
#Remove unwanted columns and rows in gene expression data
exp1 = exp_mut.drop(["Entrez_Gene_Id"], axis=1) 
exp1.dropna(inplace=True)
exp1.set_index("Hugo_Symbol", inplace=True)

#Invert the rows and columns of the data
exp1 = exp1.T

#Rename column name
exp1['Sample Identifier'] = exp1.index

exp1.head(10)

Hugo_Symbol,ACVRL1,AFF2,AGMO,AGTR2,AHNAK,AHNAK2,AKAP9,AKT1,AKT2,ALK,...,THSD7A,TP53,TTYH1,UBR5,USH2A,USP28,USP9X,UTRN,ZFP36L1,Sample Identifier
MB-0362,-0.5410,-0.6422,-0.9923,0.3742,-0.8728,0.7118,-0.5119,0.7617,1.0383,-0.0388,...,0.6323,0.1783,-0.4134,-2.5059,-0.5777,-1.0270,0.2693,0.7628,-1.3851,MB-0362
MB-0346,-0.3123,-0.0426,-0.3136,1.6764,-0.8483,0.2720,0.8209,1.8395,-0.2203,-0.3941,...,0.7708,1.5374,0.1055,-0.9736,-0.0007,-0.3306,1.1136,-0.7678,-1.9300,MB-0346
MB-0386,-0.0132,-0.1310,0.0881,0.2763,0.9198,-0.0192,-0.0318,-0.8309,-0.3195,0.3622,...,1.0874,0.2028,-0.4457,-1.0260,0.1022,-0.9569,-0.3321,0.5739,-1.3617,MB-0386
MB-0574,-0.7274,-0.2717,-0.5359,0.5059,-0.4719,-1.0170,-1.9398,-0.9817,4.0436,0.2576,...,0.0928,1.7971,-0.3272,-0.3902,0.3891,-0.5684,1.8919,0.0802,-1.1539,MB-0574
MB-0185,-1.1754,-1.2299,0.1550,0.2532,-0.8634,0.0493,0.4155,-0.5072,-0.7181,-0.9115,...,2.5150,0.5995,-0.4196,-0.9876,-0.6642,-0.2926,-1.1574,0.0751,-1.3973,MB-0185
MB-0503,0.2561,0.2915,-1.4104,0.5655,1.1923,-0.2230,-0.1753,-1.6085,1.1387,-0.7178,...,0.4360,-0.0851,-0.5554,-2.0520,0.0336,-1.5118,0.8636,-0.5426,0.4543,MB-0503
MB-0641,2.0242,0.0248,1.7006,2.2020,0.7532,0.5998,-0.1579,-0.8184,-0.8277,-0.5271,...,0.7027,0.2108,-0.1475,-2.2396,-0.5083,0.0016,-0.3306,0.3240,-0.8728,MB-0641
MB-0201,0.7213,0.5369,-0.0739,0.2421,0.3335,0.6806,0.9436,-0.0162,0.1326,-0.9742,...,-0.2751,1.0117,0.1166,-1.2259,1.1706,1.9597,-1.2128,-0.1225,-0.9160,MB-0201
MB-0218,-0.5113,0.0183,-0.3336,-1.0218,0.1457,-0.3696,0.4752,-0.0041,-1.4868,0.2310,...,-1.3153,-0.7464,-0.8788,-1.1005,-1.5685,-1.7529,-0.1957,-1.2402,0.0780,MB-0218
MB-0316,-0.7176,0.2026,-0.2631,-0.0101,-1.5826,-0.7066,-0.3032,-1.0916,0.8728,0.8301,...,-0.4412,1.8684,2.4389,-0.7539,-0.1452,-0.4162,0.1507,-0.8108,0.1575,MB-0316


In [11]:
#Isolate the pam50 breast cancer subtype column from the dataset
pam50 = pam50[["#Patient Identifier", "Pam50 + Claudin-low subtype"]]
pam50 = pam50.iloc[4:,]
pam50.rename(columns={'#Patient Identifier':'Sample Identifier'}, inplace=True)

pam50.head(10)

,Sample Identifier,Pam50 + Claudin-low subtype
4,MB-0000,claudin-low
5,MB-0002,LumA
6,MB-0005,LumB
7,MB-0006,LumB
8,MB-0008,LumB
9,MB-0010,LumB
10,MB-0014,LumB
11,MB-0020,Normal
12,MB-0022,claudin-low
13,MB-0025,NaN


In [12]:
#Count number of NAs for pam50/ caludin-low subtype
pam50.isna().sum()

Sample Identifier                0
Pam50 + Claudin-low subtype    529
dtype: int64

In [13]:
#Merge the gene expression and clinical datasets by symbol id
merged = pd.merge(clin1, pam50, how='outer', on='Sample Identifier').dropna()

#Merging the finalised dataset to include molecular subtype
merged = pd.merge(merged, exp1, how="outer", on="Sample Identifier").dropna()

merged.head(10)

,Sample Identifier,ER Status,HER2 Status,PR Status,Pam50 + Claudin-low subtype,ACVRL1,AFF2,AGMO,AGTR2,AHNAK,...,THADA,THSD7A,TP53,TTYH1,UBR5,USH2A,USP28,USP9X,UTRN,ZFP36L1
0,MB-0000,Positive,Negative,Negative,claudin-low,2.8018,0.9356,0.3067,-0.8504,1.8229,...,0.8474,4.0070,0.3481,2.0412,-1.5903,-1.0283,-0.0265,-0.0654,1.6603,0.3433
1,MB-0002,Positive,Negative,Positive,LumA,-1.8067,0.4817,-0.9456,-1.8446,1.6662,...,-0.5858,-0.6132,-0.0176,-0.7278,0.5215,-1.3180,0.1930,0.3214,0.1909,1.4831
2,MB-0005,Positive,Negative,Positive,LumB,-1.1869,0.0417,0.1168,0.9700,-0.0187,...,-1.4430,-1.3823,0.5126,-0.3872,0.5039,0.0779,2.3398,0.0411,-0.5709,2.7944
3,MB-0006,Positive,Negative,Positive,LumB,-1.7275,-0.2383,0.4058,1.4637,0.3075,...,-1.2363,0.2257,1.6748,-0.3071,1.6148,0.4038,1.5291,0.3613,-0.1076,2.0545
4,MB-0008,Positive,Negative,Positive,LumB,-0.8278,0.9725,0.9789,0.8657,-0.7646,...,-0.3349,-0.2765,0.3461,-0.1481,3.1054,-0.0013,-1.1404,0.6028,-0.7553,-0.0943
5,MB-0010,Positive,Negative,Positive,LumB,-0.2928,0.4498,-1.5594,-1.2046,0.3134,...,0.7349,-0.5363,-1.9502,-0.5007,0.3851,0.6921,-0.1630,-0.1507,0.0531,0.3152
6,MB-0014,Positive,Negative,Positive,LumB,-0.4881,0.8056,0.1276,0.3097,0.2816,...,-2.9184,0.4022,-0.5173,-0.3617,1.1951,0.3073,0.6322,-0.8498,-0.2708,2.0346
7,MB-0020,Negative,Negative,Negative,Normal,1.4061,0.7822,-0.0682,-0.2505,0.1113,...,-0.3789,1.0815,-0.4536,0.1065,-0.1990,-0.8381,-0.7641,0.2345,0.5474,0.2053
8,MB-0022,Positive,Negative,Negative,claudin-low,0.1605,0.3599,1.3598,0.8040,-0.4784,...,-1.9817,1.0986,-0.0853,-0.0966,2.4955,0.6978,0.9179,-0.8934,-1.1711,2.7166
9,MB-0028,Positive,Negative,Negative,LumB,-1.4534,1.5150,-0.4839,1.2135,-0.4467,...,-0.6995,0.7623,0.0521,-0.5370,2.8367,5.5288,0.1575,-0.8704,-1.3340,-0.8548


In [14]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1980 entries, 0 to 1979
Columns: 178 entries, Sample Identifier to ZFP36L1
dtypes: float64(173), object(5)
memory usage: 2.7+ MB


In [15]:
#Output the merged dataset as a csv file
merged.to_csv("MTBC_Breast_Cancer.csv", sep='\t', index=False)